In [5]:
import os
import torch
from safetensors.torch import load_file, save_file, safe_open
from transformers import AutoConfig

torch.set_default_device('cuda:0')

/root/miniconda3/envs/vllm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# NOTE: the conversion is model specific -- it works only for Llama-2
# checkpoint_path = '/net/storage149/autofs/css22/cliu22/llama2_70bchat_gptq/gptq_output_act1_grp128_bluewiki'
checkpoint_path = '/data/models/ibm/llama-2-70b-gptq/llama-70b-gptq-act1-grp128-tp-aware'

In [7]:
# Load configs for debug assertions
config = AutoConfig.from_pretrained(checkpoint_path)
hidden_size = config.hidden_size
intermediate_size = config.intermediate_size
group_size = config.quantization_config['group_size']
bits = config.quantization_config['bits']

In [8]:
state_dict = load_file(os.path.join(checkpoint_path, 'model.safetensors.original'))

In [9]:
# print(state_dict['model.layers.9.self_attn.q_proj.g_idx'])
# print(state_dict['model.layers.9.self_attn.k_proj.g_idx'])
# print(state_dict['model.layers.9.self_attn.v_proj.g_idx'])
# print(torch.equal(state_dict['model.layers.9.self_attn.q_proj.g_idx'], state_dict['model.layers.9.self_attn.k_proj.g_idx']) 
#       and torch.equal(state_dict['model.layers.9.self_attn.q_proj.g_idx'], state_dict['model.layers.9.self_attn.v_proj.g_idx']))

In [10]:
# print(state_dict['model.layers.8.self_attn.q_proj.g_idx'])
# print(state_dict['model.layers.8.self_attn.k_proj.g_idx'])
# print(state_dict['model.layers.8.self_attn.v_proj.g_idx'])

In [11]:
# print(state_dict['model.layers.8.self_attn.q_proj.qweight'])
# print(state_dict['model.layers.8.self_attn.k_proj.qweight'])
# print(state_dict['model.layers.8.self_attn.v_proj.qweight'])

In [12]:
# print(state_dict['model.layers.8.self_attn.q_proj.scales'])
# print(state_dict['model.layers.8.self_attn.k_proj.scales'])
# print(state_dict['model.layers.8.self_attn.v_proj.scales'])

In [13]:
# print(state_dict['model.layers.8.self_attn.q_proj.qzeros'])
# print(state_dict['model.layers.8.self_attn.k_proj.qzeros'])
# print(state_dict['model.layers.8.self_attn.v_proj.qzeros'])
# torch.equal(state_dict['model.layers.8.self_attn.q_proj.qzeros'], state_dict['model.layers.8.self_attn.k_proj.qzeros'])

In [14]:
print(state_dict['model.layers.9.mlp.up_proj.g_idx'])
print(state_dict['model.layers.9.mlp.gate_proj.g_idx'])
torch.equal(state_dict['model.layers.9.mlp.up_proj.g_idx'], state_dict['model.layers.9.mlp.gate_proj.g_idx'])

tensor([ 6, 59, 18,  ..., 56, 12, 38], device='cuda:0', dtype=torch.int32)
tensor([ 6, 59, 18,  ..., 56, 12, 38], device='cuda:0', dtype=torch.int32)


True

In [15]:
# Show some examples
state_dict['model.layers.9.mlp.up_proj.qweight']

tensor([[ 1504102265,  1989637763, -1450472794,  ...,  2007455285,
         -1197963625,  2034726259],
        [ 1150651017,  1719163430,  1806150534,  ..., -2070165900,
          -861243228,  -912684695],
        [ 1735931273, -1440978586, -2056701570,  ..., -1738033288,
         -1735742587, -1717276089],
        ...,
        [ -390814618,  1986534773, -1720158039,  ..., -1445767350,
         -1754683419,  1771063605],
        [-1264268426, -2021487965,  2025306986,  ...,  1186240105,
          1421317545, -1852012662],
        [-1179142536,  -979978346,  1161476001,  ..., -1420248477,
           968382584, -2033743465]], device='cuda:0', dtype=torch.int32)

In [16]:
state_dict['model.layers.9.mlp.up_proj.g_idx']

tensor([ 6, 59, 18,  ..., 56, 12, 38], device='cuda:0', dtype=torch.int32)

In [17]:
print(state_dict['model.layers.9.mlp.up_proj.qweight'].shape)
print(state_dict['model.layers.9.mlp.up_proj.g_idx'].shape)
print(state_dict['model.layers.9.mlp.up_proj.scales'].shape)
print(state_dict['model.layers.9.mlp.up_proj.qzeros'].shape)

torch.Size([1024, 28672])
torch.Size([8192])
torch.Size([64, 28672])
torch.Size([64, 3584])


In [18]:
# Shuffle columns of scales
def shuffle_and_replace_scales(state_dict, scales_name, col_perm):
    scales = state_dict[scales_name]
    assert len(col_perm) == scales.shape[1]
    
    shuffled_scales = scales[:,col_perm]
    state_dict[scales_name] = shuffled_scales

def unpack_shuffle_repack_and_replace_qzeros(state_dict, bits, qzeros_name, col_perm):
    qzeros = state_dict[qzeros_name]
    mask = 2**bits - 1
    pack_size = 32 // bits
    assert len(col_perm) == qzeros.shape[1] * pack_size
    
    #unpack
    unpacked_qzeros = torch.zeros((qzeros.shape[0], qzeros.shape[1]*pack_size), dtype=torch.int)
    for i in range(pack_size):
        unpacked_qzeros[:, i::pack_size] = (qzeros >> (i*bits)) & (mask) 

    # shuffle
    shuffled_qzeros = unpacked_qzeros[:,col_perm]

    # repack
    packed_qzeros = torch.zeros_like(qzeros)
    for i in range(pack_size):
        packed_qzeros |= (shuffled_qzeros[:, i::pack_size] & mask) << (i*bits)     
    
    state_dict[qzeros_name] = packed_qzeros
    
def shuffle_and_replace_qweight(state_dict, bits, group_size, qweight_name, g_idx_name=None, next_g_idx_name=None, stable=False):
    qweight = state_dict[qweight_name]

    # unpack qweight
    mask = 2**bits - 1
    pack_size = 32 // bits
    unpacked_qweight = torch.zeros((qweight.shape[0]*pack_size, qweight.shape[1]), dtype=torch.int)
    for i in range(pack_size):
        unpacked_qweight[i::pack_size] = (qweight >> (i*bits)) & (mask) 

    # reorder rows conditionally
    if not (g_idx_name is None):
        g_idx = state_dict[g_idx_name]
        row_perm = torch.argsort(g_idx, stable=stable)
        unpacked_qweight = unpacked_qweight[row_perm]
    
    # reorder columns conditionally
    if not (next_g_idx_name is None):
        next_g_idx = state_dict[next_g_idx_name]
        col_perm = torch.argsort(next_g_idx, stable=stable)
        unpacked_qweight = unpacked_qweight[:,col_perm]

    # pack qweight
    packed_qweight = torch.zeros_like(qweight)
    for i in range(pack_size):
        packed_qweight |= (unpacked_qweight[i::pack_size] & mask) << (i*bits) 

    # replace qweight with new reordered one in state_dict
    print(f'replacing {qweight_name}')
    state_dict[qweight_name] = packed_qweight
    
    if not (g_idx_name is None):
        print(f'replacing {g_idx_name}')
        state_dict[g_idx_name] = torch.arange(0, len(g_idx), dtype=torch.int) // group_size 

In [19]:
# Process column parallel layers first
# Reorder their columns only 
for name in state_dict.keys():
    if 'mlp.gate_proj.qweight' in name: # col-parallel layer
        qweight_name = name
        next_g_idx_name = name.replace('gate_proj.qweight', 'down_proj.g_idx')
        scales_name = name.replace('qweight', 'scales')
        qzeros_name = name.replace('qweight', 'qzeros')
        next_g_idx = state_dict[next_g_idx_name]
        perm = torch.argsort(next_g_idx)
        
        shuffle_and_replace_scales(state_dict, scales_name, perm)
        unpack_shuffle_repack_and_replace_qzeros(state_dict, bits, qzeros_name, perm)
        shuffle_and_replace_qweight(state_dict, bits, group_size, qweight_name, next_g_idx_name=next_g_idx_name)
    elif 'mlp.up_proj.qweight' in name: # col-parallel layer
        qweight_name = name
        next_g_idx_name = name.replace('up_proj.qweight', 'down_proj.g_idx')
        scales_name = name.replace('qweight', 'scales')
        qzeros_name = name.replace('qweight', 'qzeros')
        next_g_idx = state_dict[next_g_idx_name]
        perm = torch.argsort(next_g_idx)
        
        shuffle_and_replace_scales(state_dict, scales_name, perm)
        unpack_shuffle_repack_and_replace_qzeros(state_dict, bits, qzeros_name, perm)
        shuffle_and_replace_qweight(state_dict, bits, group_size, qweight_name, next_g_idx_name=next_g_idx_name)

# Process row parallel layers after all column parallel layers because of dependency on g_idx
for name in state_dict.keys():        
    if 'mlp.down_proj.qweight' in name: # row-parallel layer
        qweight_name = name
        g_idx_name = name.replace('qweight', 'g_idx')
        shuffle_and_replace_qweight(state_dict, bits, group_size, qweight_name, g_idx_name)

replacing model.layers.0.mlp.gate_proj.qweight
replacing model.layers.0.mlp.up_proj.qweight
replacing model.layers.1.mlp.gate_proj.qweight
replacing model.layers.1.mlp.up_proj.qweight
replacing model.layers.10.mlp.gate_proj.qweight
replacing model.layers.10.mlp.up_proj.qweight
replacing model.layers.11.mlp.gate_proj.qweight
replacing model.layers.11.mlp.up_proj.qweight
replacing model.layers.12.mlp.gate_proj.qweight
replacing model.layers.12.mlp.up_proj.qweight
replacing model.layers.13.mlp.gate_proj.qweight
replacing model.layers.13.mlp.up_proj.qweight
replacing model.layers.14.mlp.gate_proj.qweight
replacing model.layers.14.mlp.up_proj.qweight
replacing model.layers.15.mlp.gate_proj.qweight
replacing model.layers.15.mlp.up_proj.qweight
replacing model.layers.16.mlp.gate_proj.qweight
replacing model.layers.16.mlp.up_proj.qweight
replacing model.layers.17.mlp.gate_proj.qweight
replacing model.layers.17.mlp.up_proj.qweight
replacing model.layers.18.mlp.gate_proj.qweight
replacing model.

In [20]:
save_file(state_dict, os.path.join(checkpoint_path, 'model.safetensors.reordered'))